In [1]:
import os, cv2, geojson
import numpy as np
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import cm, colors
from pyproj import Proj
UTM_10_PROJ = Proj("+proj=utm +zone=10N, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

In [2]:
def stddev_above_threshold(x,y,window_size, threshold,im):
    r = im[x*window_size:(x+1)*window_size, y*window_size:(y+1)*window_size, 0:3]
    return r.std() > threshold
def compressed_green(x,y,window_size,green_mask):
    r = green_mask[x*window_size:(x+1)*window_size, y*window_size:(y+1)*window_size]
    return r.mean() > 128
def convert_to_lon_lat(polygon_raw, x_res, y_res, utm_10_top_left_coord):
    polygon_processed = [[x[0][0]*x_res+utm_10_top_left_coord[0],x[0][1]*y_res+utm_10_top_left_coord[1]] for x in polygon_raw]
    projected = [UTM_10_PROJ(x[0], x[1], inverse=True) for x in polygon_processed]
    head = projected[0]
    projected.append(head)
    return projected
# size of image
width = 10000
# granularity of subimages
g = 20
# number of subimages
n = int(width / g)
stddev_threshold = 12
dark_green = (55/360*255,0, 0)
light_green = (180/360*255, 255, 150)

In [3]:
os.chdir("/home/jonathan/ubc/capstone/orthophoto/2014")
image = cv2.imread('483E_5454N.tif')

In [4]:
kernel = np.ones((g,g),np.float32)/(g*g)
blurred_image = cv2.filter2D(image,-1,kernel)
hsv_blurred_image = cv2.cvtColor(blurred_image, cv2.COLOR_RGB2HSV)
mask = cv2.inRange(hsv_blurred_image, dark_green, light_green)

In [5]:
blurred_image = ndimage.gaussian_filter(image, sigma=2)
stddev_array = np.array([stddev_above_threshold(i//n,i%n,g,stddev_threshold,blurred_image) for i in range(0,n**2)]).reshape(n,n)
compressed_green_array = np.array([compressed_green(i//n,i%n,g,mask) for i in range(0,n**2)]).reshape(n,n)
green_stddev_array = np.array([compressed_green_array[i//n,i%n] and stddev_array[i//n,i%n] for i in range(0,n**2)]).reshape(n,n)
close_img = ndimage.binary_closing(green_stddev_array)
open_img = ndimage.binary_opening(close_img)

In [6]:
thresh = np.array(open_img*255,dtype=np.uint8)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
len(contours)

169

In [9]:
feature_list = []
for c in contours:
    lon_lat = convert_to_lon_lat(c*g,0.1,-0.1,[483000.050000,5454999.950000])
    feature_list.append(geojson.Feature(geometry=geojson.Polygon([lon_lat])))
feature_collection = geojson.FeatureCollection(feature_list)

with open("polygons2.geojson", mode = "w") as out_file:
    geojson.dump(feature_collection,out_file)